In [ ]:
from pathlib import Path
import shutil

In [ ]:
!pip install pandas

In [ ]:
!pip install matplotlib

In [ ]:
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.models import load_model

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

In [ ]:
def small_cnn(input_shape = (256, 256, 1), lr = 1e-4, factor = 16):
    img_input = layers.Input(input_shape)
#     X = layers.Conv2D(factor, 3, padding = 'same', activation = 'relu')(img_input)
#     X = layers.MaxPooling2D(pool_size = (2, 2))(X)
    X = layers.Conv2D(factor*2, 3, padding = 'same', activation = 'relu')(img_input)
    X = layers.MaxPooling2D(pool_size = (2, 2))(X)
    X = layers.Conv2D(factor*4, 3, padding = 'same', activation = 'relu')(X)
    X = layers.MaxPooling2D(pool_size = (2, 2))(X)
    X = layers.Conv2D(factor*8, 3, padding = 'same', activation = 'relu')(X)
    X = layers.MaxPooling2D(pool_size = (2, 2))(X)
    X = layers.Conv2D(factor*16, 3, padding = 'same', activation = 'relu')(X)
    X = layers.MaxPooling2D(pool_size = (2, 2))(X)
    X = layers.Flatten()(X)
    X = layers.Dense(128, activation = 'relu')(X)
    X = layers.Dropout(rate=0.5)(X)
    X = layers.Dense(1)(X)
    X = layers.Activation('sigmoid', dtype='float32', name='predictions')(X)
    model = models.Model(inputs = img_input, outputs = X)
    model.compile(optimizer = Adam(lr), loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [ ]:
batch_size = 16
target_size = (256, 256)
color_mode = 'grayscale'
dataset = '/raid/audio_detection/audio_detection/data'
dataset_1= 'new'
train_datagen = ImageDataGenerator(rescale=1/255)
val_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)
print(os.getcwd())
print(os.chdir(dataset))
print(os.getcwd())


train_generator = train_datagen.flow_from_directory(f'{dataset}/train/', target_size=target_size, batch_size=batch_size, class_mode='binary', color_mode=color_mode, classes = ['real','fake'])
val_generator = val_datagen.flow_from_directory(f'{dataset}/val/', target_size=target_size, batch_size=batch_size, class_mode='binary', color_mode=color_mode, classes = ['real','fake'])
test_generator = test_datagen.flow_from_directory(f'{dataset}/test/', target_size=target_size, batch_size=batch_size, class_mode='binary', color_mode=color_mode, classes = ['real','fake'])


for attempt in range(1):
    print(os.getcwd())
    model_path = f'models/{dataset}_{attempt}'
    print(os.getcwd())
    Path(model_path).mkdir(parents=True, exist_ok = True)
    checkpointer = ModelCheckpoint(model_path +'5_layers_lr_10e-4_epochs:{epoch:02d}_acc:{val_accuracy:.4f}.h5', monitor = 'val_loss', save_best_only = True, verbose = 0, mode = 'auto')
    #checkpointer = ModelCheckpoint(filepath='model._epochs:{epoch:02d}_acc:{val_accuracy:.4f}.h5',monitor = 'val_loss', save_best_only = True, verbose = 0, mode = 'auto')
    earlystopper = EarlyStopping(monitor = 'val_loss', patience = 5, verbose = 0, mode = 'auto')
    reduceLR = ReduceLROnPlateau(monitor = 'val_loss', factor = 1/np.sqrt(10), patience = 3, cooldown = 1, verbose = 0, mode = 'auto')
    model = small_cnn((256,256,1))
    history = model.fit(train_generator, steps_per_epoch = len(train_generator), verbose = 2, epochs = 50, callbacks = [checkpointer, earlystopper, reduceLR], validation_data=val_generator, validation_steps = len(val_generator))
    print(model.evaluate(test_generator, steps = 2*len(test_generator)))
    
    ## added later on
    results = model.evaluate(val_generator,steps=2*len(val_generator),verbose=0)
    print('Development Set:')
    print(f'Loss: {results[0]:.4f}\nAccuracy: {results[1]:.4f}')
    results = model.evaluate(test_generator,steps=2*len(test_generator),verbose=0)
    print('Evaluation Set:')
    print(f'Loss: {results[0]:.4f}\nAccuracy: {results[1]: .4f}')

In [ ]:
model.summary()

In [ ]:
hist_df = pd.DataFrame(history.history) 

In [ ]:
hist_df

In [ ]:
hist_csv_file = 'history_5_layers_lr_10e-4_layers.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)